In [1]:
import os, sys
import torch
import datasets
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments,
    GenerationConfig
)
from peft import PeftModel, LoraConfig, prepare_model_for_kbit_training, get_peft_model

2024-05-10 15:25:27.198961: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 15:25:28.184829: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
### config ###
model_id = "/data/qm/huggingface/Llama-2-7b-chat-hf"
max_length = 4096
device_map = "auto"
batch_size = 128
micro_batch_size = 32
gradient_accumulation_steps = batch_size // micro_batch_size

# nf4" use a symmetric quantization scheme with 4 bits precision
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# load model from huggingface
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    use_cache=True,
    device_map=device_map
)

# load tokenizer from huggingface
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model.device.type

'cuda'

In [9]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot, always able to provide friendly and helpful answers."
    },
    {
        "role": "user",
        "content": "What athlete from the 1992 U.S. Olympic 'Dream Team' had a role in the movie 'Space Jam'?"
    },
    {
        "role": "assistant",
        "content": "The athlete from the 1992 U.S. Olympic 'Dream Team' who had a role in the movie 'Space Jam' is Michael Jordan. He played the role of the basketball player \"Lucas North\" in the film, which was released in 1996. The movie was based on a fictional basketball team called the \"Space Jam,\" which was created by Warner Bros. And NBC Universal for the release of the basketball movie \"Space Jam\" in 1996."
    },
    {
        "role": "user",
        "content": "Can you give me more information about that?"
    }
]
model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
generated_ids = model.generate(model_inputs, max_new_tokens=512, temperature=0.7)
result = tokenizer.batch_decode(generated_ids)
print(result)

['<s> [INST] <<SYS>>\nYou are a friendly chatbot, always able to provide friendly and helpful answers.\n<</SYS>>\n\nWhat athlete from the 1992 U.S. Olympic \'Dream Team\' had a role in the movie \'Space Jam\'? [/INST] The athlete from the 1992 U.S. Olympic \'Dream Team\' who had a role in the movie \'Space Jam\' is Michael Jordan. He played the role of the basketball player "Lucas North" in the film, which was released in 1996. The movie was based on a fictional basketball team called the "Space Jam," which was created by Warner Bros. And NBC Universal for the release of the basketball movie "Space Jam" in 1996. </s><s> [INST] Can you give me more information about that? [/INST] Of course! The 1992 U.S. Olympic "Dream Team" was a men\'s basketball team that represented the United States at the 1992 Summer Olympics in Barcelona, Spain. The team was composed of some of the best professional basketball players in the world at the time, including Michael Jordan, Magic Johnson, Larry Bird, 

In [5]:
### generate ###
prompt = "Write me a poem about Singapore."
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
generate_ids = model.generate(inputs.input_ids, max_new_tokens=100, do_sample=True,
                              temperature=0.7, top_k=50,
                              top_p=0.95)
print('\nAnswer:\n', tokenizer.decode(generate_ids[0]))


Answer:
 <s> Write me a poem about Singapore. Hinweis: I will provide the poem with the theme of "Singapore" and you will write a poem based on that theme.

Title: The Lion City's Pride

Singapore, a land of contrasts,
Where East meets West in vibrant hues.
From bustling streets to quiet retreats,
The Lion City's charm never loses its zest.

From Gardens by the Bay to Chinatown


In [10]:
### generate ###
prompt = '''
Your goal is to split the task given below into a number of subtasks, with the required number of subtasks being no less than two and no more than five.
Please return a JSON object.
The original task and the paths to the files involved in the subtasks should be stored in "File Information".Each file should contain a "file path" field and a "description" field.
The contents of the subtasks should be stored in "Subtasks".Each subtask should contain an 'id' field and a 'task description' field.

For example,
Task: I need to extract the water zone of XiHu district, only keep the water zone
      the green band tif of XiHu district is in $GREEN_BAND_FILEPATH.
      the nir band tif of XiHu district is in $NIR_BAND_FILEPATH.
      please save the final result in $OUTPUT_PATH.

Output:
{{
  "Subtasks":[
    {{
      "id": 1,
      "task description": "Calculate the ndwi of XiHu district."
    }},
    {{
      "id": 2,
      "task description": "Extract the water zone."
    }}
  ],
  "File Information":[
    {{
      "file path": "$GREEN_BAND_FILEPATH",
      "description": "the green band tif of XiHu district."
    }},
    {{
      "file path": "$NIR_BAND_FILEPATH",
      "description": "the nir band tif of XiHu district."
    }},
    {{
      "file path": "$OUTPUT_PATH",
      "description": "Used to save the final result."
    }}
  ]
}}


Task: There are road shapefile without time and speed information, administrative region shapefile without area imformation and hospitals point shapefile.
  Please create service areas for tertiary hospitals within 0-20 minutes and 20-45 minutes driving distance
  and Calculate the percentage of the total area covered by the service areas of top-tier hospitals within 0-20 minutes and 20-45 minutes in each administrative region.

  Here is the data you will need:
  shape_file_path='data/road.shp',
  hospitals_path='data/hospitals.shp',
  administrative_region_path='data/administrative.shp'.

  The meaning of the 'fclass' field for main roads and the average vehicle speed are as follows:
  motorway: 100km/h;
  motoway_link: 40km/h;
  trunk: 70km/h;
  primary: 55km/h;
  primary_link:40km/h;
  secondary_link:40km/h;
  trunk:40k5/h;
  tertiary_link:40km/h;


Begin!
Describe your subtasks with rich details. Do not response anything unrelated to the subtask or file information.
Remeber to ouput a JSON object.
'''
inputs = tokenizer(prompt, return_tensors="pt")
generate_ids = model.generate(inputs.input_ids, max_length=1024)
print('\nAnswer:\n', tokenizer.decode(generate_ids[0]))


Answer:
 <s> 
Your goal is to split the task given below into a number of subtasks, with the required number of subtasks being no less than two and no more than five.
Please return a JSON object.
The original task and the paths to the files involved in the subtasks should be stored in "File Information".Each file should contain a "file path" field and a "description" field.
The contents of the subtasks should be stored in "Subtasks".Each subtask should contain an 'id' field and a 'task description' field.

For example,
Task: I need to extract the water zone of XiHu district, only keep the water zone
      the green band tif of XiHu district is in $GREEN_BAND_FILEPATH.
      the nir band tif of XiHu district is in $NIR_BAND_FILEPATH.
      please save the final result in $OUTPUT_PATH.

Output:
{{
  "Subtasks":[
    {{
      "id": 1,
      "task description": "Calculate the ndwi of XiHu district."
    }},
    {{
      "id": 2,
      "task description": "Extract the water zone."
    }}
 